# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print (child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print ('* ' + element.find('name').text + ':'),
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print (capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [5]:
from xml.etree import ElementTree as ET
import numpy as np
import pandas as pd

In [6]:
document = ET.parse( './data/mondial_database.xml' )

In [7]:
source = document.getroot()

In [8]:
def cities(elem):
    for e in elem.findall('city'):
        yield e
    for pro in elem.findall('province'):
        for x in pro.findall('city'):
            yield x

country_d = {}
city_d= {}

ethnic_list = []
popl_list = []

for country in source.iter('country'):
    country_name = country.find('name').text
    country_d[country_name] = {}
    country_d[country_name]['name'] = country_name
    country_d[country_name]['car_code'] = country.attrib['car_code']
    try:
        country_d[country_name]['infant_mortality'] = country.find('infant_mortality').text
    except:
        country_d[country_name]['infant_mortality'] = np.nan
    
    country_d[country_name]['area'] = country.attrib.get('area', np.nan)
    country_d[country_name]['capital'] = country.attrib.get('capital', np.nan)
    
# country_d1 = pd.DataFrame.from_dict(country_d, orient='index')
# country_d1 = pd.DataFrame(country_d).transpose() 

    # 10 cities with the largest population.To answer this question, 
    # let us populate the country information in ethnic_list    
    for e in country.findall('ethnicgroup'):
        ethnic = {}
        ethnic_name = e.text
        ethnic['name'] = ethnic_name
        ethnic['country'] = country_name
        ethnic['percentage'] = e.attrib.get('percentage',np.nan)
        ethnic_list.append(ethnic)
        
    # Let us populate ethnic group info in city_dict
    for e1 in cities(country):
        city = {}
        cityname = e1.find('name').text 
        city['name'] = cityname
        pop_years = []
        
        for elem in e1.findall('population'):
            pop_years.append(int(elem.attrib['year']))
            key = 'population_' + elem.attrib['year']
            city[key] = elem.text
        try:
            late_year = max(pop_years)
            late_yearkey = 'population_' + str(late_year)
            city['population_latest'] = city[late_yearkey]
        except:
            city['population_latest'] = np.nan
        city_d[cityname] = city  
        
    # Let us populate country info in pop_list
    for elem in country.findall('population'):
        popl = {}
        popl['population'] = int(elem.text)
        popl['est_type'] = elem.attrib.get('measured', np.nan)
        popl['year'] = elem.attrib.get('year', np.nan)
        popl['country'] = country_name
        popl_list.append(popl)


In [9]:
#  10 countries with the lowest infant mortality rates
country_d1 = pd.DataFrame.from_dict(country_d, orient='index')
country_d1.infant_mortality = country_d1.infant_mortality.astype(float)
country_d1.sort_values(by='infant_mortality', ascending=True).head(10).infant_mortality


Monaco            1.81
Japan             2.13
Norway            2.48
Bermuda           2.48
Singapore         2.53
Sweden            2.60
Czech Republic    2.63
Hong Kong         2.73
Macao             3.13
Iceland           3.15
Name: infant_mortality, dtype: float64

In [10]:
# 10 cities with the largest population
c_largepop = pd.DataFrame.from_dict(city_d, orient='index')
c_largepop = c_largepop.dropna(subset=['population_latest'],how='all')
c_largepop.population_latest = c_largepop.population_latest.astype(int)
c_largepop.sort_values(by='population_latest', ascending=False).head(10).population_latest

Shanghai     22315474
Istanbul     13710512
Mumbai       12442373
Moskva       11979529
Beijing      11716620
São Paulo    11152344
Tianjin      11090314
Guangzhou    11071424
Delhi        11034555
Shenzhen     10358381
Name: population_latest, dtype: int32

In [17]:
# Population estimate of the countries based on the latest census

popl_est = pd.DataFrame(popl_list)
#popl_est_census = popl_est[popl_est.est_type == 'census']
popl_est_best = popl_est.ix[popl_est.groupby('country',as_index=False)['year'].idxmax(axis='year')]
popl_est_best.head()

,country,est_type,population,year
570,Afghanistan,estimate,26023100,2013
8,Albania,census,2800138,2011
1741,Algeria,est.,37062820,2010
1389,American Samoa,census,55519,2010
63,Andorra,admin.,78115,2011


In [18]:
#  10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
df = pd.DataFrame(ethnic_list)
df_ethnic = pd.merge(df, popl_est_best, on='country')
df_ethnic.percentage = df_ethnic.percentage.astype(float)
df_ethnic['ethn_pop'] = ((df_ethnic.percentage / 100.00) * df_ethnic.population).astype(int)
ethn_sum = df_ethnic.groupby('name').sum()

# 10 Ethnic groups with their respective population size sorted in descending order
ethn_sum.ethn_pop.sort_values(ascending=False).head(10)

name
Han Chinese    1245058800
Indo-Aryan      871815583
European        494872201
African         318325104
Dravidian       302713744
Mestizo         157734349
Bengali         146776916
Russian         131856989
Japanese        126534212
Malay           121993548
Name: ethn_pop, dtype: int32

In [13]:
# Let us pull the country_d and set the 'car_code' as an index
df_carcode = pd.DataFrame.from_dict(country_d, orient='index').reset_index().set_index('car_code')
df_carcode.head(3)

,index,name,infant_mortality,area,capital
car_code,,,,,
AFG,Afghanistan,Afghanistan,117.23,647500,cty-Afghanistan-Kabul
AL,Albania,Albania,13.19,28750,cty-Albania-Tirane
DZ,Algeria,Algeria,21.76,2381740,cty-Algeria-Algiers


In [14]:
# name and country of a) longest river
dict_river = {}

for r in source.iter('river'):
    river = {}
    river_id = r.attrib['id']
    river['name'] = r.find('name').text
    river['ccodes'] = r.attrib.get('country',np.nan)
    try:
        river['length'] = r.find('length').text
    except:
        river['length'] = np.nan
        
    
    country_names = []
    for cc in r.attrib['country'].split(' '):
        country_names.append(df_carcode.loc[cc]['index'])
    
    river['country_names'] = country_names
    dict_river[river_id] = river
    
df_river = pd.DataFrame.from_dict(dict_river, orient='index')
df_river.length = df_river.length.astype(float)
df_river.sort_values(by='length', ascending=False).head(10).rename(columns = {'ccodes':'country_abbrevation'})

,name,country_abbrevation,length,country_names
river-Amazonas,Amazonas,CO BR PE,6448.0,"[Colombia, Brazil, Peru]"
river-Jangtse,Jangtse,CN,6380.0,[China]
river-Hwangho,Hwangho,CN,4845.0,[China]
river-Lena,Lena,R,4400.0,[Russia]
river-Zaire,Zaire,RCB ZRE,4374.0,"[Congo, Zaire]"
river-Mekong,Mekong,CN LAO THA K VN,4350.0,"[China, Laos, Thailand, Cambodia, Vietnam]"
river-Irtysch,Irtysch,R KAZ CN,4248.0,"[Russia, Kazakhstan, China]"
river-Niger,Niger,RMM RN WAN RG,4184.0,"[Mali, Niger, Nigeria, Guinea]"
river-Missouri_River,Missouri,USA,4130.0,[United States]
river-Jenissej,Jenissej,R,4092.0,[Russia]


In [15]:
# Name and Country of b) largest lake

dict_lake = {}
for l in source.iter('lake'):
    lake = {}
    lake_id = l.attrib['id']
    lake['name'] = l.find('name').text
    lake['ccodes'] = l.attrib.get('country')
    
    try:
        lake['area'] = float(l.find('area').text)
    except:
        lake['area'] = np.nan
        
    
    country_names = []
    for cc in l.attrib['country'].split(' '):
        country_names.append(df_carcode.loc[cc]['index'])
    
    lake['country_names'] = country_names
    dict_lake[lake_id] = lake
    
df_lake = pd.DataFrame.from_dict(dict_lake, orient='index')
df_lake.area = df_lake.area.astype(float)
df_lake.sort_values(by='area', ascending=False).head(1).rename(columns = {'ccodes':'country_abbrevation'})

,name,country_abbrevation,area,country_names
lake-KaspischesMeer,Caspian Sea,R AZ KAZ IR TM,386400.0,"[Russia, Azerbaijan, Kazakhstan, Iran, Turkmen..."


In [16]:
# Name and Country of c) airport at highest elevation

dict_air = {}
for a in source.iter('airport'):
    airport = {}
    air_id = a.attrib.get('iatacode')
    airport['name'] = a.find('name').text
    airport['ccodes'] = a.attrib.get('country')
    
    try:
        airport['elevation'] = float(a.find('elevation').text)
    except:
        airport['elevation'] = np.nan
        
    
    country_names = []
    for cc in a.attrib['country'].split(' '):
        country_names.append(df_carcode.loc[cc]['index'])
        
    airport['country_names'] = country_names
    dict_air[air_id] = airport

df_air = pd.DataFrame.from_dict(dict_air, orient='index')
df_air.area = df_air.elevation.astype(float)
df_air.sort_values(by='elevation', ascending=False).head(1).rename(columns = {'ccodes':'country_abbrevation'})
    

,name,country_abbrevation,elevation,country_names
LPB,El Alto Intl,BOL,4063.0,[Bolivia]
